In [1]:
import pandas as pd
import openai
import numpy as np
import time
import tiktoken
openai.api_key = ''
import concurrent.futures
import math


In [29]:
df = pd.read_csv("classified_data_complete.csv")
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

In [6]:
df.isna().sum()

date          0
title         0
text          0
governor      0
tokens        0
Relevant    260
clean        13
dtype: int64

In [8]:
df_na = df[df["Relevant"].isna()]
df_na

,date,title,text,governor,tokens,Relevant,clean
174,2022-09-28,The New Landscape for Banking Competition,"decades, we've seen credit unions expand thei...",Governor Michelle W. Bowman,1000,NaN,No Topic
721,2019-08-05,Delivering Fast Payments for All,"small or challenging to reach, and with compe...",Governor Lael Brainard,904,NaN,No Topic
890,2018-11-16,Beginning Stress Testingâs New Chapter,"Professor Scott, and our hosts from Harvard an...",Vice Chairman for Supervision Randal K. Quarles,1000,NaN,No Topic
913,2018-10-15,Community Investment in Denver,business models. We should consider whether a...,Governor Lael Brainard,331,NaN,No Topic
940,2018-07-18,Getting It Right: Factors for Tailoring Superv...,applying enhanced prudential standards for fi...,Vice Chairman for Supervision Randal K. Quarles,1000,NaN,No Topic
...,...,...,...,...,...,...,...
5277,2011-06-22,FOMC Press Conference number 2,we need to make sure that that’s well underst...,CHAIRMAN BERNANKE,500,NaN,No Topic
5283,2011-06-22,FOMC Press Conference number 2,factor in—in growth and employment in an econ...,CHAIRMAN BERNANKE,500,NaN,No Topic
5296,2011-04-27,FOMC Press Conference number 1,", and we’ll have—we’ll be close to our medium-...",CHAIRMAN BERNANKE,500,NaN,No Topic
5299,2011-04-27,FOMC Press Conference number 1,announce that they would restrain production ...,CHAIRMAN BERNANKE,500,NaN,No Topic


In [40]:
# Function to make a single API call to OpenAI's API
def make_api_call(speech):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "system", "content": "You are a knowledgeable assistant specialized in identifying economic aspects mentioned in given texts. Provide the topic mentioned among these: Inflation, Economic Growth, Employment, Interest Rates."},
                {"role": "user", "content": "Speech: 'The current fiscal policy has led to unprecedented growth. GDP has risen 3% in the last quarter alone, indicating our economy is on a solid trajectory.' \n Topic (Inflation, Economic Growth, Employment, Interest Rates, No Topic):"},
                {"role": "assistant", "content": "Economic Growth"},
                {"role": "user", "content": "Speech: 'Our employment rates have never been better. More people are joining the workforce every day, contributing to the nation's prosperity' \n Topic (Inflation, Economic Growth, Employment, Interest Rates, No Topic):"},
                {"role": "assistant", "content": "Employment"},
                {"role": "user", "content": "Speech: 'There has been a lot of discussion recently regarding new technology trends shaping the banking industry.' \n Topic (Inflation, Economic Growth, Employment, Interest Rates, No Topic):"},
                {"role": "assistant", "content": "No Topic"},
                {"role": "user", "content": "Speech: 'The Federal Reserve has decided to increase the interest rates by 0.25% in an effort to curb the rising inflation and stabilize the economy.'\n Topic (Inflation, Economic Growth, Employment, Interest Rates, No Topic):"},
                {"role": "assistant", "content": "Inflation, Interest Rates"},
                {"role": "user", "content": f"""Speech: "{speech}" \n Topic (Inflation, Economic Growth, Employment, Interest Rates, No Topic):"""}],
            temperature=0.3,
            max_tokens=10
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"An error occurred while making API call: {e}")
        return None



def process_dataframe(df, x=20, y=0.8):
    """
    Processes the given DataFrame by making API calls in batches.
    
    :param df: DataFrame to process.
    :param x: Number of API calls to make at once.
    :param y: Time to wait between batches in minutes.
    """
    # Calculate the number of batches
    num_batches = math.ceil(len(df) / x)

    # Loop through the DataFrame in batches
    for i in range(num_batches):
        start_idx = i * x
        end_idx = start_idx + x
        
        # Select the rows where "Relevant" is NaN
        
        rows_to_update = df.iloc[start_idx:end_idx][df['Relevant'].isna().iloc[start_idx:end_idx]]
        
        # Break the loop if there are no more NaN values
        if rows_to_update.empty:
            break

        # Making X API calls concurrently
        with concurrent.futures.ThreadPoolExecutor(max_workers=x) as executor:
            results = list(executor.map(make_api_call, rows_to_update['text']))

        # Insert the results into the 'Relevant' column of the DataFrame
        df.loc[rows_to_update.index, 'Relevant'] = results
        
        # Wait for y minutes before the next batch
        df.to_csv("classification_nans.csv", index=False)
        print(f"currently classifying rows:{start_idx} to {end_idx}")
        time.sleep(y * 60)

    # Return the processed DataFrame
    return df


In [11]:
processed_df = process_dataframe(df_na)

An error occurred while making API call: Rate limit reached for default-gpt-3.5-turbo in organization org-YcAgrXsREPxCOIWgXL0Yxnop on tokens per min. Limit: 90000 / min. Current: 88337 / min. Contact us through our help center at help.openai.com if you continue to have issues.
An error occurred while making API call: Rate limit reached for default-gpt-3.5-turbo in organization org-YcAgrXsREPxCOIWgXL0Yxnop on tokens per min. Limit: 90000 / min. Current: 89843 / min. Contact us through our help center at help.openai.com if you continue to have issues.
An error occurred while making API call: Rate limit reached for default-gpt-3.5-turbo in organization org-YcAgrXsREPxCOIWgXL0Yxnop on tokens per min. Limit: 90000 / min. Current: 89761 / min. Contact us through our help center at help.openai.com if you continue to have issues.
An error occurred while making API call: Rate limit reached for default-gpt-3.5-turbo in organization org-YcAgrXsREPxCOIWgXL0Yxnop on tokens per min. Limit: 90000 / m

In [14]:
processed_df["clean"] = np.nan

/var/folders/sw/lwscf3w57bz06p9m9g2_5q1r0000gn/T/ipykernel_74335/2327706222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df["clean"] = np.nan


In [15]:
#cleaning pipeline
# Function to make a single API call to OpenAI's API
def make_api_call(speech):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "system", "content": """You are performing a data cleaning task. 
                 Please clean the input and use only these 5 labels, use comma as separator: 
                 Inflation, Economic Growth, Employment, Interest Rates, No Topic"""},
                {"role": "user", "content": """ 'the mentioned categories are inflation, and rates'\n Labels:"""},
                {"role": "assistant", "content": "Inflation, Interest Rates"},
                {"role": "user", "content": "'Labour market, Economy growing'\n Labels:"},
                {"role": "assistant", "content": "Employment, Economic Growth"},
                {"role": "user", "content": "'Inflation'\n Labels:"},
                {"role": "assistant", "content": "Inflation"},
                {"role": "user", "content": f"""'{speech}' \n Labels:"""}],
            temperature=1,
            max_tokens=10
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"An error occurred while making API call: {e}")
        return None



def clean_dataframe(df, x=100, y=0.5):
    """
    Processes the given DataFrame by making API calls in batches.
    
    :param df: DataFrame to process.
    :param x: Number of API calls to make at once.
    :param y: Time to wait between batches in minutes.
    """
    # Calculate the number of batches
    num_batches = math.ceil(len(df) / x)

    # Loop through the DataFrame in batches
    for i in range(num_batches):
        start_idx = i * x
        end_idx = start_idx + x
        
        # Select the rows where "Relevant" is NaN
        
        rows_to_update = df.iloc[start_idx:end_idx][df['clean'].isna().iloc[start_idx:end_idx]]
        
        # Break the loop if there are no more NaN values
        if rows_to_update.empty:
            break

        # Making X API calls concurrently
        with concurrent.futures.ThreadPoolExecutor(max_workers=x) as executor:
            results = list(executor.map(make_api_call, rows_to_update['Relevant']))

        # Insert the results into the 'Relevant' column of the DataFrame
        df.loc[rows_to_update.index, 'clean'] = results
        
        # Wait for y minutes before the next batch
        df.to_csv("cleaning_experiment.csv", index=False)
        print(f"currently classifying rows:{start_idx} to {end_idx}")
        time.sleep(y * 60)

    # Return the processed DataFrame
    return df

In [18]:
categories = ['Inflation', 'Interest Rates', 'Economic Growth', 'Employment', 'No Topic']

def clean_categories(row):
    # Check if the row is a string
    if isinstance(row, str):
        # Split the categories and strip whitespace
        row_categories = [category.strip() for category in row.split(',')]
        # Filter out any categories not in the acceptable list
        row_categories = [category for category in row_categories if category in categories]
        # If 'No Topic' is not the only category, remove it
        if 'No Topic' in row_categories and len(row_categories) > 1:
            row_categories.remove('No Topic')
        # Join the categories back together
        return ', '.join(row_categories)
    else:
        # If the row is not a string, return 'No Topic'
        return 'No Topic'

# Apply the function to the 'clean' column
processed_df_clean['clean'] = processed_df_clean['clean'].apply(clean_categories)

# Display the first few rows of the dataset after cleaning
processed_df_clean.head()

/var/folders/sw/lwscf3w57bz06p9m9g2_5q1r0000gn/T/ipykernel_74335/934405874.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df_clean['clean'] = processed_df_clean['clean'].apply(clean_categories)


,date,title,text,governor,tokens,Relevant,clean
174,2022-09-28,The New Landscape for Banking Competition,"decades, we've seen credit unions expand thei...",Governor Michelle W. Bowman,1000,None,No Topic
721,2019-08-05,Delivering Fast Payments for All,"small or challenging to reach, and with compe...",Governor Lael Brainard,904,No Topic,No Topic
890,2018-11-16,Beginning Stress Testingâs New Chapter,"Professor Scott, and our hosts from Harvard an...",Vice Chairman for Supervision Randal K. Quarles,1000,None,No Topic
913,2018-10-15,Community Investment in Denver,business models. We should consider whether a...,Governor Lael Brainard,331,No Topic,No Topic
940,2018-07-18,Getting It Right: Factors for Tailoring Superv...,applying enhanced prudential standards for fi...,Vice Chairman for Supervision Randal K. Quarles,1000,No Topic,No Topic
